# Example of DOV search methods for lithologische beschrijvingen

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_lithologische_beschrijvingen.ipynb)

## Use cases:
* Select records in a bbox
* Select records in a bbox with selected properties
* Select records in a municipality
* Get records using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect

In [2]:
import pydov

## Get information about code base

In [3]:
from pydov.search.interpretaties import LithologischeBeschrijvingenSearch
ip_litho = LithologischeBeschrijvingenSearch()

# information about the HydrogeologischeStratigrafie type (In Dutch):
ip_litho.get_description()

'Wanneer een boring wordt uitgevoerd wordt er een beschrijving gemaakt van het materiaal dat men aangeboord heeft. Men beschrijft de textuur, het soort materiaal, de kleur, mineralen, fossielen, ... per diepte-interval. Dit kan zowel op het veld gebeuren als achteraf indien de stalen bewaard worden. Deze beschrijvingen kunnen gedaan worden door bijvoorbeeld de boormeester of door een geoloog. Meestal wordt de boring beschreven per diepte-interval waarover men hetzelfde materiaal herkent. De mate van detail van de beschrijving kan afhankelijk zijn van het doeleinde waarvoor men de boring uitvoert of wie de beschrijving doet: is het om een geologische kaart te maken, dienen er monsters genomen te worden voor laboratoriumanalyses, ... In DOV worden de beschrijvingen van ontsluitingen boven het aardoppervlak, aangeduid als boringen met diepte 0 m, beschreven van 0 m tot 0 m.'

In [4]:
# information about the available fields for a HydrogeologischeStratigrafie object
fields = ip_litho.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
diepte_laag_van
diepte_laag_tot
beschrijving


In [5]:
# print information for a certain field
fields['beschrijving']

{'name': 'beschrijving',
 'type': 'string',
 'definition': 'Lithologische beschrijving van de laag in vrije tekst (onbeperkt in lengte)',
 'notnull': False,
 'query': False,
 'cost': 10}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [6]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'Type_proef':
fields['Type_proef']

{'name': 'Type_proef',
 'definition': "Het type van de proef waarvan de lithologische beschrijving een interpretatie is. Bij een lithogische beschrijving zal dit altijd 'Boring' zijn.",
 'type': 'string',
 'notnull': False,
 'query': True,
 'cost': 1,
 'values': {'Boring': 'De lithologische beschrijving is een interpretatie van een boring.'}}

## Try-out of use cases

### Select interpretations in a bbox

In [7]:
from pydov.util.location import Within, Box

# Get all lithological descriptions in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_litho.search(location=Within(Box(152145, 204930, 153150, 206935)))
df = df[df.beschrijving.notnull()]
df.head()

[000/020] ....................


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1895...,goed,152195.0,205501.0,11.0,0.00,2.00,Sable limoneux
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1895...,goed,152195.0,205501.0,11.0,2.00,2.70,"Sable verdâtre,fossilifère"
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1879...,goed,152558.0,205063.0,9.5,0.00,0.30,Terre végétale argileuse
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1879...,goed,152558.0,205063.0,9.5,0.30,1.15,Sable jaune de plus en plus argileux
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1879...,goed,152558.0,205063.0,9.5,1.15,1.20,"Sable jaune de plus en plus argileux, argileux..."


### Select interpretations in a bbox with selected properties

In [8]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
methods

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In [9]:
from owslib.fes import PropertyIsGreaterThanOrEqualTo

The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [10]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the hydrogeological interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='betrouwbaarheid_interpretatie', literal='goed')
df = ip_litho.search(location=Within(Box(153145, 206930, 153150, 206935)),
                        query=query
                       )
df.head()

[000/001] c


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,0.0,0.5,"bruinachtige zandhoudende leem, geen kalk"
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,0.5,1.0,bleekgrijze leem
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,1.0,1.5,bruingroen leemhoudend fijn zand
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,1.5,2.0,bleekgrijs leemhoudend fijn zand
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,2.0,2.5,"idem, met veenresten en kalkhoudend"


### Select interpretations in a municipality

In [11]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Aartselaar')
df = ip_litho.search(query=query)
df.head()

[000/055] ..................................................
[050/055] .....


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1870...,goed,151724.0,203063.0,16.0,0.0,0.0,Terre végétale.
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1870...,goed,151724.0,203063.0,16.0,0.0,1.0,"Sable jaune, mouvant à la base"
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1870...,goed,151724.0,203063.0,16.0,1.0,5.0,"Sable jaune, mouvant à la base glauconifére, p..."
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1870...,goed,151724.0,203063.0,16.0,5.0,10.2,"Argile verte compacte, avec pyrites"
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1870...,goed,151724.0,203063.0,16.0,10.2,21.0,Un separia.


### Visualize results

Using Folium, we can display the results of our search on a map.

In [12]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [13]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [14]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    # limit marker size for folium (:10)
    folium.Marker(loclist[loc], popup=df['beschrijving'][loc][:10]).add_to(marker_cluster)
fmap